In [ ]:
#ID01 - Image Attributes for ID01

# Pending
# - Merge with ID01

In [3]:
import os
import pandas as pd
import numpy as np 
from glob import iglob
import matplotlib.pyplot as plt
#pd.__version__

header_list = ['Date', 'DateStr','TimeStr' ,'Open', 'High', 'Low', 'Close', 'Null1', 'Null2', 'Null3', 'Null4']
path = r'/home/parag/devArea/pyStock/Data/Consolidated/*banknifty.txt'
BNF = pd.concat((pd.read_csv(f, header=None, sep=',', names=header_list) for f in iglob(path, recursive=True)), ignore_index=True)
BNF['Date']=(pd.to_datetime(BNF.DateStr.astype(str) + ' ' + BNF.TimeStr.astype(str), format='%Y%m%d %H:%M'))
BNFs = BNF.drop(['DateStr', 'TimeStr', 'Null1', 'Null2', 'Null3', 'Null4'], axis=1)
BNFs.index = BNFs["Date"]

In [4]:
# Configuration Data
CFG_orb_candles = 15 # val * 1min candles
CFG_match_timestamp = "20210119"

In [5]:
ID_01_Scan = pd.DataFrame(columns=['Date', 'Prev Close', 'Gap', 'EoD Delta', 'Today Open', 'EoD Close', 'Eod Close in Gap', 'Day Range', 'Day Volatility'])

# Copy todays data
today = BNFs.loc[BNFs.index.strftime("%Y%m%d") == CFG_match_timestamp]
today.index = today["Date"]
day_low = today["Low"].min()
day_high = today["High"].max()

# Determine ORB
loc_arry = BNFs.index.get_loc(CFG_match_timestamp)
orb      = BNFs.iloc[loc_arry[0]:loc_arry[0]+CFG_orb_candles]
orb_high = orb["Low"].min()
orb_low = orb["High"].max()


prev_day_close = BNFs["Close"].iloc[loc_arry[0]-1]
next_day_open = BNFs["Open"].iloc[loc_arry[loc_arry.size-1]+1]
today_open = BNFs["Open"].iloc[loc_arry[0]]
today_close = BNFs["Close"].iloc[loc_arry[loc_arry.size-1]]

if today_close >  today_open :
    gapState = "Above Gap"
elif today_close < prev_day_close :    
    gapState = "Below Gap"
else :
    gapState = "In the Gap"

data = [{'Date' : CFG_match_timestamp,
         'Gap' : (today_open - prev_day_close).astype(int), 
         'EoD Delta' : (today_close - today_open).astype(int), 
         'Today Open' : today_open ,
         'Prev Close' : prev_day_close ,
         'EoD Close' : today_close ,
         'Eod Close in Gap' : gapState,
         'Day Range': day_high - day_low,
         'Day Volatility' : 0}]

ID_01_Scan = ID_01_Scan.append(data,ignore_index=False,sort=False)

ID_01_Scan.head()
#orb.tail()


,Date,Prev Close,Gap,EoD Delta,Today Open,EoD Close,Eod Close in Gap,Day Range,Day Volatility
0,20210119,31811.75,271,341,32082.9,32424.85,Above Gap,601.0,0


In [2]:
#print(BNFs["Close"].iloc[loc_arry[loc_arry.size-1]])